In [35]:
import os

class Country():
    """Country class which specifies the (top 10) ranking of a country.
    Optionally gives the score this country achieved in the semifinal."""
    def __init__(self, country, ranking_list, semifinal_score=0):
        self.country = country
        self.ranking_list = ranking_list
        self.semifinal_score = semifinal_score

    def get_ranking_list(self):
        return self.ranking_list

    def get_semifinal_score(self):
        return self.semifinal_score

    def get_ith_ranking(self, i):
        return self.ranking_list[i-1]

class Rule():
    """Rule class consisting of a scoring vector and a vector length.
    Scoring vector is specified from largest to smallest score.
    Will be expanded once the thesis gets to more advanced
    material."""
    def __init__(self, scoring_vector, vector_length):
        self.scoring_vector = scoring_vector
        self.vector_length = vector_length

    def get_scoring_vector(self):
        return self.scoring_vector

    def get_vector_length(self):
        return self.vector_length

class Contest():
    """Contest class includes everything to compute the original outcome
    of a certain year's ESC. Later will also be able to solve the problem(s)
    specified in the thesis proposal."""
    def __init__(self, country_list, voting_rule, nr_of_semifinals=0):
        self.country_list = country_list
        self.voting_rule = voting_rule
        self.nr_of_countries = len(country_list)
        self.nr_of_semifinals = nr_of_semifinals
        
    def compute_result(self, rule=None):
        """Computes the result and outputs a dict with each country that scored
        more than 0 points."""
        result = dict()
        if rule is None:
            rule_vector = self.voting_rule.get_scoring_vector()
            rule_length = self.voting_rule.get_vector_length()
        else:
            rule_vector = rule.get_scoring_vector()
            rule_length = rule.get_vector_length()
        for country in self.country_list:
            if isinstance(country, Country2015) and rule is None:
                jury_list = country.get_ranking_list_jury()
                tele_list = country.get_ranking_list_tele()

                rank = 0
                for c in jury_list:
                    if rank + 1 > rule_length:
                        break
                    if c not in result:
                        result[c] = 0
                    result[c] += rule_vector[rank]
                    rank += 1

                rank = 0
                for c in tele_list:
                    if rank + 1 > rule_length:
                        break
                    if c not in result:
                        result[c] = 0
                    result[c] += rule_vector[rank]
                    rank += 1
            else:
                ranking_list = country.get_ranking_list()
                rank = 0
                for c in ranking_list:
                    if rank + 1 > rule_length:
                        break
                    if c not in result:
                        result[c] = 0
                    result[c] += rule_vector[rank]
                    rank += 1

        return result

    def print_result(self, rule=None):
        """Prints the result in a ranking list format."""
        result = self.compute_result(rule)
        for element in sorted(result, key=result.get, reverse=True):
            print(element, result[element])
        print()


In [31]:
import csv

def import_scores(range1, range2, file_name = 'votes.csv'):
    """ Reads and puts the votes for the finals
    from range1 to range2 in a dictionary. This dict has 2 levels:
    Top level: key = year, value = dict of scoring countries
    Bottom level: key = scoring country, value = ranked list of preferences (top 10)"""
    f = open(file_name, "r")
    ff = csv.reader(f)

    ESC = dict()
    for year in range(range1, range2):
        ESC[str(year)] = dict()

    for row in ff:
        if row[0] == 'year':
            continue
        if int(row[0]) not in range(range1, range2) or row[1] != 'final' or row[6] == '0':
            continue
        if row[2] not in ESC[row[0]]:
            ESC[row[0]][row[2]] = dict()
        ESC[row[0]][row[2]][row[3]] = int(row[6])
        
    scores_to_rankings(ESC)
    return ESC

def scores_to_rankings(ESC_dict):
    """ ESC dict contains a year dict, which contains country dicts with a dict
    using scored_country/score tuples. This function turns that into a list
    ranking the countries instead. """
    for year in ESC_dict:
        for country in ESC_dict[year]:
            score_dict = ESC_dict[year][country]
            ranking_list = []
            for scored_country in sorted(score_dict, key=score_dict.get, reverse=True):
                ranking_list.append(scored_country)
            ESC_dict[year][country] = ranking_list
    pass

def dict_to_classes(d, voting_rule):
    """Takes as input a dict outputted by import_scores() and a scoring rule,
    and returns a dict with objects of class Contest."""
    ESC_classified = dict()
    for year in range(1975,2016):
        ESC_classified[str(year)] = None
        
    for year in d:
        country_list = []
        for country in d[year]:
            new_country = Country(country, d[year][country])
            country_list.append(new_country)
        ESC_classified[year] = Contest(country_list, voting_rule)
    
    return ESC_classified

In [3]:
rule_1975_2015 = Rule([12,10,8,7,6,5,4,3,2,1], 10)

ESC_classified = dict_to_classes(import_scores(1975, 2016), rule_1975_2015)
ESC_classified['1975'].print_result()

nl 152
gb 138
it 115
fr 91
lu 84
ch 77
fi 74
se 72
ie 68
es 53
il 40
mt 32
yu 22
mc 22
be 17
pt 16
de 15
no 11
tr 3



In [4]:
rule_1962 = Rule([3,2,1], 3)
rule_1963 = Rule([5,4,3,2,1], 5)
rule_1964_66 = Rule([5,3,1], 3)

ESC_old = dict_to_classes(import_scores(1962, 1967), rule_1964_66)

ESC_old['1962'].print_result()
ESC_old['1963'].print_result()
ESC_old['1964'].print_result()
ESC_old['1965'].print_result()
ESC_old['1966'].print_result()

fr 40
mc 20
lu 17
yu 15
gb 15
de 13
se 6
fi 6
it 4
no 3
ch 3
dk 2

ch 31
dk 31
it 24
gb 15
mc 14
fr 14
at 7
lu 4
be 3
de 1

it 49
gb 17
mc 15
fr 14
lu 14
at 11
fi 9
no 6
dk 4
nl 2
be 2
es 1

lu 32
gb 25
fr 22
at 16
it 15
ie 11
dk 10
ch 8
mc 7
se 6
nl 5
yu 2
no 1
pt 1

at 31
se 16
no 15
ie 14
be 14
ch 12
yu 9
es 9
gb 8
lu 7
de 7
fi 7
pt 6
dk 4
nl 2
fr 1



In [40]:
def import_scores_2015_and_beyond(file_name = 'votes.csv'):
    """ Hardcoded function. Reads and puts the votes for the finals
    from 2016 to 2019 in three dictionaries. Dicts have 2 levels:
    Top level: key = year, value = dict of scoring countries
    Bottom level: key = scoring country, value = ranked list of preferences (top 10)
    The dicts for the jury votes and televotes contain a ranking,
    the dict for the total score contains scores."""
    f = open(file_name, "r")
    ff = csv.reader(f)

    ESC_jury, ESC_tele, ESC_total = dict(), dict(), dict()

    for year in range(2016,2020):
        ESC_jury[str(year)], ESC_tele[str(year)] = dict(), dict()
        ESC_total[str(year)] = dict()

    for row in ff:
        if row[0] == 'year':
            continue
        if int(row[0]) not in range(2016,2020) or row[1] != 'final' or row[6] == '0':
            continue
        if row[2] not in ESC_jury[row[0]] and row[8] != 0:
            ESC_jury[row[0]][row[2]] = dict()
        if row[2] not in ESC_tele[row[0]] and row[7] != 0:
            ESC_tele[row[0]][row[2]] = dict()
        if row[2] not in ESC_total[row[0]]:
            ESC_total[row[0]][row[2]] = dict()
        if row[8] != 0:
            ESC_jury[row[0]][row[2]][row[3]] = int(row[8])
        if row[7] != 0:
            ESC_tele[row[0]][row[2]][row[3]] = int(row[7])
        ESC_total[row[0]][row[2]][row[3]] = int(row[6])

    scores_to_rankings(ESC_jury)
    scores_to_rankings(ESC_tele)

    return ESC_jury, ESC_tele, ESC_total

def pre_2015_ify(jury, tele, total, voting_rule=Rule([12,10,8,7,6,5,4,3,2,1], 10)):
    """Converts the post 2015 total score (which has ties) to a 1975-2012 ranking.
    To break ties, the jury vote is checked: the country which is ranked higher
    in the jury vote wins the tie."""
    result = dict()
    rule_vector = voting_rule.get_scoring_vector()
    rule_length = voting_rule.get_vector_length()
    
    def swap_positions(l, elem1, elem2):
        pos1, pos2 = l.index(elem1), l.index(elem2)
        l[pos1], l[pos2] = l[pos2], l[pos1] 
    
    def bubble_sort_tuple_list(l):
        for i in range(len(l) - 1):
            if l[i][1] == l[i + 1][1]:
                if tele.index(l[i+1][0]) < tele.index(l[i][0]):
                    swap_positions(tuple_list, tuple_list[i+1], tuple_list[i])
    
    pre_2015_ranking = []
    tuple_list = []
    
    for element in sorted(total, key=total.get, reverse=True):
        tuple_list.append((element, total[element]))
        
    for i in range(5):
        bubble_sort_tuple_list(tuple_list)
    
    ranking_list = []
    for i in range(len(tuple_list)):
        ranking_list.append(tuple_list[i][0])
        
    return ranking_list
        
def dicts_to_2015_classes(d_jury, d_tele, d_total, voting_rule):
    """Takes as input two dicts outputted by import_scores_2015_and_beyond() and
    a scoring rule, and returns a dict with objects of class Contest."""
    ESC_classified = dict()
    for year in range(2016,2020):
        ESC_classified[str(year)] = None

    for year in d_jury:
        country_list = []
        for country in d_jury[year]:
            new_country = Country2015(country,
                                      d_jury[year][country], 
                                      d_tele[year][country],
                                      d_total[year][country])
            country_list.append(new_country)
        ESC_classified[year] = Contest(country_list, voting_rule)

    return ESC_classified

class Country2015(Country):
    """ranking_list contains a ranking list as specified by pre_2015_ify().
    The original scores can be computed with the scoring rule and jury and tele rankings."""
    def __init__(self, country, jury, tele, total, semifinal_score=0):
        self.country = country
        self.ranking_list_jury = jury
        self.ranking_list_tele = tele
        self.ranking_list = pre_2015_ify(jury, tele, total)
        self.semifinal_score = semifinal_score
        
    def get_ranking_list_jury(self):
        return self.ranking_list_jury
    
    def get_ranking_list_tele(self):
        return self.ranking_list_tele
        

In [41]:
rule_1975_2015 = Rule([12,10,8,7,6,5,4,3,2,1], 10)
d_jury, d_tele, d_total = import_scores_2015_and_beyond()
ESC_classified_2015 = dicts_to_2015_classes(d_jury, d_tele, d_total, rule_1975_2015)

In [59]:
rule_1962 = Rule([3,2,1], 3)
rule_1963 = Rule([5,4,3,2,1], 5)
rule_1964_66 = Rule([5,3,1], 3)
rule_1975_2015 = Rule([12,10,8,7,6,5,4,3,2,1], 10)
ESC_classified_2015['2019'].print_result(rule_1975_2015)

nl 293
it 271
ru 202
no 190
ch 186
se 171
mk 150
az 148
au 136
is 112
cz 70
cy 49
mt 47
al 47
dk 43
rs 42
si 37
sm 34
fr 33
gr 32
ee 31
es 18
by 15
il 13
de 8



In [8]:
def compute_scores_for_rules(ESC_dict, rules):
    """Takes as input a dict of ESC's and a list of rules.
    Outputs a list with a dict of ESC's for each rule.
    Optionally prints the scores for each rule."""
    
    for rule in rules:
        for contest in ESC_dict:
#             print("### " + str(contest) + " normal ###")
#             ESC_dict[contest].print_result()
            print("### " + str(contest) + " new rule ###")
            ESC_dict[contest].print_result(rule)
        
    

In [9]:
rules_list = [Rule([3,2,1], 3), Rule([5,4,3,2,1], 5), Rule([5,3,1], 3)]
compute_scores_for_rules(ESC_classified, rules_list)

### 1975 new rule ###
nl 27
gb 22
it 15
fr 11
fi 10
lu 8
ch 6
ie 5
pt 3
il 2
se 2
es 1
mt 1
de 1

### 1976 new rule ###
gb 33
fr 25
it 10
at 7
ch 6
mc 6
be 6
ie 6
il 4
pt 3
gr 1
nl 1

### 1977 new rule ###
gb 22
fr 20
ie 20
mc 10
gr 9
ch 7
be 6
fi 4
nl 3
de 3
il 2
lu 1
it 1

### 1978 new rule ###
il 27
be 17
fr 13
ie 12
mc 11
lu 9
de 8
gr 5
ch 4
es 4
it 4
nl 3
se 2
gb 1

### 1979 new rule ###
il 23
es 20
de 13
fr 13
dk 9
ch 6
gb 6
ie 5
no 4
gr 4
nl 3
it 3
pt 2
lu 2
fi 1

### 1980 new rule ###
ie 25
de 19
nl 15
ch 13
it 10
gb 10
se 5
pt 4
tr 4
lu 3
at 2
be 2
es 1
gr 1

### 1981 new rule ###
ch 21
fr 19
de 18
gb 17
ie 14
cy 7
se 5
es 4
dk 3
yu 3
gr 3
lu 2
il 2
pt 1
be 1

### 1982 new rule ###
de 35
ch 15
il 15
gb 8
cy 8
be 6
lu 4
es 4
se 3
at 3
yu 3
no 2
tr 1
ie 1

### 1983 new rule ###
lu 26
yu 22
il 19
se 16
de 11
gb 6
gr 6
fr 4
nl 3
no 2
at 2
fi 1
be 1
it 1

### 1984 new rule ###
ie 23
se 22
es 12
be 11
dk 10
it 9
fr 6
cy 5
gb 4
ch 3
yu 2
pt 2
tr 2
nl 1
no 1
lu 1

### 1985 new rule ##

gb 35
fr 31
ie 30
mc 15
gr 14
ch 10
be 9
fi 6
nl 4
de 3
il 3
lu 1
it 1

### 1978 new rule ###
il 41
be 28
fr 18
ie 18
mc 16
lu 15
de 12
gr 7
es 6
ch 5
it 5
nl 5
se 3
gb 1

### 1979 new rule ###
il 36
es 31
de 21
fr 20
dk 14
ch 9
gb 8
ie 7
gr 6
no 5
nl 4
it 3
pt 3
lu 3
fi 1

### 1980 new rule ###
ie 40
de 29
nl 24
ch 20
it 15
gb 14
se 7
tr 6
pt 5
lu 3
at 3
be 3
es 1
gr 1

### 1981 new rule ###
ch 33
fr 30
de 27
gb 24
ie 22
cy 10
se 8
es 6
dk 5
yu 4
gr 4
lu 3
il 2
pt 1
be 1

### 1982 new rule ###
de 56
ch 23
il 23
gb 13
cy 12
be 8
lu 5
es 5
yu 5
at 4
se 3
no 3
tr 1
ie 1

### 1983 new rule ###
lu 40
yu 34
il 29
se 23
de 16
gr 10
gb 9
fr 6
nl 5
at 3
no 2
fi 1
be 1
it 1

### 1984 new rule ###
ie 36
se 35
es 18
be 17
dk 15
it 14
fr 9
cy 8
gb 5
ch 4
tr 3
yu 2
pt 2
nl 1
no 1
lu 1

### 1985 new rule ###
no 40
de 23
it 19
se 16
ie 11
il 10
at 9
fr 8
gb 8
es 6
fi 6
tr 6
lu 4
dk 3
gr 1
cy 1

### 1986 new rule ###
be 53
ch 34
lu 23
ie 19
se 10
de 8
gb 8
tr 7
dk 6
yu 5
nl 4
fi 1
pt 1
es 1

### 1987 